In [2]:
import pandas as pd
import numpy as np

In [3]:
predictions = pd.read_csv(f"../data/predictions/predictions-{11}-{14}.csv")
# store_item = pd.read_csv(f"../data/predictions/predictions-{11}-{6}.csv").store_item
# predictions = pd.concat([predictions, store_item], axis=1)


In [4]:
predictions

,Unnamed: 0,store_item,sales,autoreg,exp_smooth,ardl,xgb_preds,prophet,neural_prophet
0,2016-07-03,1-1,35,33.967322,34.968360,28.270421,26.026503,31.430007,32.687370
1,2016-07-04,1-1,18,27.360797,27.868874,21.079731,11.948491,23.552477,24.825729
2,2016-07-05,1-1,31,26.177283,26.861003,21.245674,9.154426,26.190409,27.409180
3,2016-07-06,1-1,22,27.303443,28.344893,27.543148,10.446870,26.846605,28.147953
4,2016-07-07,1-1,33,23.602249,25.971329,29.099083,11.971621,27.095248,28.449118
...,...,...,...,...,...,...,...,...,...
272401,2017-12-27,10-50,63,54.000000,70.629780,52.203080,53.286358,60.990158,76.515198
272402,2017-12-28,10-50,59,48.333333,64.050097,60.144470,61.416860,64.923885,80.440819
272403,2017-12-29,10-50,74,41.333333,56.527002,60.437527,62.223686,69.576875,85.073769
272404,2017-12-30,10-50,62,43.666667,59.522268,69.928363,67.933410,74.064814,89.512894


In [ ]:

features = pd.read_csv("../data/tsfeatures/tsfeatures.csv")
predictions.drop(predictions.columns[0], axis=1, inplace=True)
predictions.rename(columns={list(predictions)[0]:'date'}, inplace=True)
predictions['date'] = pd.to_datetime(predictions['date'], errors='coerce')


sales = pd.DataFrame(predictions.sales)

errors = predictions.drop(predictions.columns[[0, 1, 2]], axis=1)\
    .sub(sales.sales, axis=0)\
        .divide(sales.sales, axis=0) * 100

In [4]:
from helper import create_features

In [6]:
predictions

,date,sales,autoreg,exp_smooth,ardl,xgb_preds,prophet,neural_prophet
0,NaT,35,33.967322,34.968360,28.270421,26.026503,31.430007,32.687370
1,NaT,18,27.360797,27.868874,21.079731,11.948491,23.552477,24.825729
2,NaT,31,26.177283,26.861003,21.245674,9.154426,26.190409,27.409180
3,NaT,22,27.303443,28.344893,27.543148,10.446870,26.846605,28.147953
4,NaT,33,23.602249,25.971329,29.099083,11.971621,27.095248,28.449118
...,...,...,...,...,...,...,...,...
272401,2050-10-01,63,54.000000,70.629780,52.203080,53.286358,60.990158,76.515198
272402,2050-10-01,59,48.333333,64.050097,60.144470,61.416860,64.923885,80.440819
272403,2050-10-01,74,41.333333,56.527002,60.437527,62.223686,69.576875,85.073769
272404,2050-10-01,62,43.666667,59.522268,69.928363,67.933410,74.064814,89.512894


In [5]:
model_least_err = pd.DataFrame(errors.abs().idxmin(axis=1), columns=["model_w_least_error"])

errors_df = pd.concat([predictions[["store_item", "date"]], model_least_err], axis=1)

feats = features.drop(features.columns[[0, 3, 6, 7, 8]], axis=1)\
    .rename(columns={"unique_id":"store_item"})

errors_and_feats = pd.merge(errors_df, feats, right_on="store_item", left_on= "store_item")

error_time = pd.concat([errors_and_feats, create_features(predictions)], axis=1)\
    .dropna(axis = 1, how = 'all')\
    .dropna()
    
X, y = error_time.iloc[:, 3:], error_time.iloc[:, 2]

KeyError: "['store_item'] not in index"

In [6]:
X.to_csv("../data/forecasts_for_ml/X.csv")
y.to_csv("../data/forecasts_for_ml/y.csv")